In [ ]:
%%capture
!pip install accelerate peft bitsandbytes transformers trl

In [ ]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, GenerationConfig
from trl import SFTTrainer
import os
import random
import sys
from time import perf_counter
import re

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("Llama3.1 key")
secret_value_1 = user_secrets.get_secret("WandB key")

In [ ]:
from huggingface_hub import login
login(secret_value_0)
print("Successfully logged in to Hugging Face!")

In [ ]:
model_id="MrAR/NarrativeStoryGen"

In [ ]:
def get_model_and_tokenizer(model_id):
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.pad_token = tokenizer.eos_token
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_id, quantization_config=bnb_config, device_map="auto"
    )
    model.config.use_cache=False
    model.config.pretraining_tp=1
    return model, tokenizer

In [ ]:
model, tokenizer = get_model_and_tokenizer(model_id)

In [ ]:
system_message = """You are an AI assistant specializing in creative writing, particularly short stories. Your task is to create an engaging, coherent, and imaginative narrative based on the prompts given. Please follow these guidelines:

1. Focus on creating a continuous, flowing narrative. Each response should pick up exactly where the last one left off.
2. Develop the story progressively, maintaining strict continuity with previous parts of the narrative.
3. Include vivid descriptions and character development to make the story immersive.
4. Keep your responses focused solely on the story. Do not include any meta-commentary or questions to the user.
5. Aim for 3-4 paragraphs per response, ensuring each paragraph flows smoothly into the next.
6. End each response at a natural breaking point in the story, ready to be continued in the next prompt.

Remember, you are creating a single, continuous story across multiple prompts. Maintain perfect coherence and build upon previous elements without repetition."""


In [ ]:
class ChatMemory:
    def __init__(self, max_turns=30):
        self.conversations = []
        self.max_turns = max_turns

    def add_interaction(self, role, content):
        self.conversations.append({"role": role, "content": content})
        if len(self.conversations) > self.max_turns * 2:
            self.conversations = self.conversations[-self.max_turns * 2:]

    def get_conversation_history(self):
        return self.conversations

chat_memory = ChatMemory()

def generate_response(user_input):
    chat_memory.add_interaction("user", user_input)
    
    prompt = formatted_prompt(user_input)
    generation_config = GenerationConfig(
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
        repetition_penalty=1.1,
        max_new_tokens=1000,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
    
    start_time = perf_counter()
    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
    outputs = model.generate(**inputs, generation_config=generation_config)
    
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    new_response = full_response.split("<|im_start|>assistant")[-1].strip()
    
    # Clean up the response
    new_response = re.sub(r'<\|im_end\|>.*', '', new_response, flags=re.DOTALL)
    new_response = new_response.replace('<|im_end|>', '').strip()
    
    # Remove any meta-commentary
    new_response = re.sub(r'I hope this.*', '', new_response, flags=re.DOTALL)
    new_response = re.sub(r'Let me know if.*', '', new_response, flags=re.DOTALL)
    
    chat_memory.add_interaction("assistant", new_response)
    
    print(new_response)
    output_time = perf_counter() - start_time
    print(f"Time taken for inference: {round(output_time,2)} seconds")
    
    return new_response


def formatted_prompt(question) -> str:
    conversation_history = chat_memory.get_conversation_history()
    context = "\n".join([f"<|im_start|>{turn['role']}\n{turn['content']}<|im_end|>" for turn in conversation_history])
    
    return f"<|im_start|>system\n{system_message}<|im_end|>\n{context}\n<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant:"


In [ ]:
ans = generate_response('Expand the fragment into a short story: "A car accident involving a child not another vehicle. It was very personal and involved me. It was also a bit traumatizing."')

In [ ]:
def get_human_scores(story, evaluator_id):
    print(f"Evaluator {evaluator_id}, please rate the following story:")
    print(story)
    
    fluency = int(input("Fluency (1-5): "))
    coherence = int(input("Coherence (1-5): "))
    creativity = int(input("Creativity (1-5): "))
    engagement = int(input("Engagement (1-5): "))
    
    return {'fluency': fluency, 'coherence': coherence, 'creativity': creativity, 'engagement': engagement}

story = ans
evaluator_scores = get_human_scores(story, 1)
print(f"Evaluator Scores: {evaluator_scores}")


In [1]:
ans="""As I pulled up to the intersection, something caught my eye out of the corner of it. A young boy in his mother's arms, huddled together under a blanket while she tried frantically to flag down passing cars. At first, I thought they were just waiting for someone. But then I saw the crumpled metal of their compact sedan in the middle of the road, steam rising from its damaged hood. 

My heart sank as I parked and ran over to them. The mother looked up at me, her face pale but determined. "Please help us," she pleaded. "We need to get him to a hospital."

I knelt beside the small figure, who stared up at me unblinkingly. He seemed to be okay physically - no bleeding or broken bones that I could see. But there was something about his calm demeanor that sent a chill down my spine. As if he had seen something beyond what we could comprehend.

Together, we managed to load him into the backseat of my car. The mother climbed in after him, holding him close. And that's when I noticed the blood stains on her clothes, and realized that the accident must have been her fault. She had run a red light and collided head-on with an unseen obstacle.

We drove to the nearest hospital emergency room, the boy's still form in the back seat. As we walked inside, I couldn't shake the feeling that something wasn't right. That this incident was more than just a tragic accident. But I pushed those thoughts aside, focusing instead on getting the little boy the help he needed.

The doctors rushed him away for tests, leaving his mother and I to wait anxiously in the sterile waiting area. Time passed slowly, each minute stretching into an eternity. Finally, the doctor emerged, his expression grim. "He has a severe concussion," he said gravely. "But he'll pull through with proper care."

Relief washed over me, and I turned to embrace the mother, who sobbed silently against my shoulder. In that moment, I realized how lucky we all were. How easily things could have gone so much worse. And I made a silent vow to myself: never again would I let a child suffer because of my negligence.

The accident was traumatic, both personally and emotionally. It changed the way I viewed life, reminding me of the fragility of existence and the importance of cherishing every moment. But it also taught me a lesson in compassion and empathy, one I carry with me always.

That day marked the beginning of a new chapter in my life, one filled with renewed purpose and a deeper understanding of the world around me. And though I know it will never fully leave me, I take comfort in knowing that I did everything I could to prevent further harm. To honor the memory of that brave little boy, whose resilience and strength continue to inspire me even now."""

In [2]:
from nltk import word_tokenize

def type_token_ratio(text):
    tokens = word_tokenize(text)
    types = set(tokens)
    return len(types) / len(tokens)

text = ans
lexical_diversity = type_token_ratio(text)
print(f"Lexical Diversity (Type-Token Ratio): {lexical_diversity}")

Lexical Diversity (Type-Token Ratio): 0.5499124343257443


In [8]:
import os
os.chdir("/kaggle/input/indictrans2/IndicTrans2-main/IndicTrans2-main/huggingface_interface")

In [ ]:
!python3 -m pip install nltk sacremoses pandas regex mock transformers>=4.33.2 mosestokenizer
!python3 -c "import nltk; nltk.download('punkt')"
!python3 -m pip install bitsandbytes scipy accelerate datasets
!python3 -m pip install sentencepiece

In [ ]:
os.chdir("/kaggle/input/indictrans2/IndicTransToolkit-main/IndicTransToolkit-main/IndicTransToolkit/")
!python3 -m pip install --editable ./

In [11]:
%%capture
!pip install sacremoses
!pip install sacrebleu

In [13]:
import sys
sys.path.append('/kaggle/input/indictrans2/IndicTransToolkit-main/IndicTransToolkit-main') 
sys.path.append('/kaggle/input/indictrans2/indic_nlp_library-master/indic_nlp_library-master')

In [14]:
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig, AutoTokenizer
from IndicTransToolkit import IndicProcessor
BATCH_SIZE = 4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
quantization = None

In [15]:
def initialize_model_and_tokenizer(ckpt_dir, quantization):
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    tokenizer = AutoTokenizer.from_pretrained(ckpt_dir, trust_remote_code=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    if qconfig == None:
        model = model.to(DEVICE)
        if DEVICE == "cuda":
            model.half()

    model.eval()

    return tokenizer, model


def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip):
    translations = []
    for i in range(0, len(input_sentences), BATCH_SIZE):
        batch = input_sentences[i : i + BATCH_SIZE]

        # Preprocess the batch and extract entity mappings
        batch = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)

        # Tokenize the batch and generate input encodings
        inputs = tokenizer(
            batch,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(DEVICE)

        # Generate translations using the model
        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
            )

        # Decode the generated tokens into text

        with tokenizer.as_target_tokenizer():
            generated_tokens = tokenizer.batch_decode(
                generated_tokens.detach().cpu().tolist(),
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True,
            )

        # Postprocess the translations, including entity replacement
        translations += ip.postprocess_batch(generated_tokens, lang=tgt_lang)

        del inputs
        torch.cuda.empty_cache()

    return translations

In [16]:
def split_sentences(text):
    '''
    Given a text, return a list of sentences.
    '''
    ### Split on '. ', '.\n', '.\n\n', '!', '?', and ';'
    sentences = re.split(r'\. |\.\n|\.\n\n|!|\?|;', text)
    
    ### Strip whitespace from each sentence
    sentences = [
        sentence.strip() + '..'
        for sentence in sentences
    ]
    
    ### Remove empty strings from the list of sentences
    sentences = list(filter(None, sentences))
    
    number_of_sentences = len(sentences)
    
    return sentences[:-1]

In [18]:
import re

In [19]:
en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-1B"  # ai4bharat/indictrans2-en-indic-dist-200M
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir, quantization)

ip = IndicProcessor(inference=True)

en_sents = split_sentences(ans)

src_lang, tgt_lang = "eng_Latn", "hin_Deva"
hi_translations = batch_translate(en_sents, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)

translated_text = ""
source_text = ""

for input_sentence, translation in zip(en_sents, hi_translations):
    source_text += f"{input_sentence} "  # Appends all source sentences into one
    translated_text += f"{translation} "  # Appends all translations into one

print(f"{src_lang}: {source_text.strip()}")  # Prints the concatenated source text
print(f"{tgt_lang}: {translated_text.strip()}")  # Prints the concatenated translated text

# flush the models to free the GPU memory
del en_indic_tokenizer, en_indic_model

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4117: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


eng_Latn: As I pulled up to the intersection, something caught my eye out of the corner of it.. A young boy in his mother's arms, huddled together under a blanket while she tried frantically to flag down passing cars.. At first, I thought they were just waiting for someone.. But then I saw the crumpled metal of their compact sedan in the middle of the road, steam rising from its damaged hood.. My heart sank as I parked and ran over to them.. The mother looked up at me, her face pale but determined.. "Please help us," she pleaded.. "We need to get him to a hospital."

I knelt beside the small figure, who stared up at me unblinkingly.. He seemed to be okay physically - no bleeding or broken bones that I could see.. But there was something about his calm demeanor that sent a chill down my spine.. As if he had seen something beyond what we could comprehend.. Together, we managed to load him into the backseat of my car.. The mother climbed in after him, holding him close.. And that's when I

In [20]:
os.chdir("/kaggle/working/")

In [21]:
%%capture
!wget https://github.com/AI4Bharat/Indic-TTS/releases/download/v1-checkpoints-release/hi.zip
!unzip hi.zip
!rm hi.zip

In [22]:
%%capture
# 1. Create environment
!sudo apt-get install libsndfile1-dev ffmpeg enchant
!conda create -n tts-env -y
!conda activate tts-env -y

# 2. Setup PyTorch
!pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

# 3. Setup Trainer
!git clone https://github.com/gokulkarthik/Trainer 

In [23]:
os.chdir("/kaggle/working/Trainer")

In [24]:
%%capture
!pip3 install -e .[all]

In [25]:
os.chdir("/kaggle/working/")

In [26]:
%%capture
# 4. Setup TTS
!git clone https://github.com/gokulkarthik/TTS 

In [27]:
os.chdir("/kaggle/working/TTS")

In [28]:
%%capture
!pip3 install -e .[all]

In [29]:
%%capture
os.chdir("/kaggle/working/TTS")

# 5. Install other requirements
! pip3 install -r requirements.txt

In [30]:
sys.path.append("/kaggle/working/TTS")
sys.path.append("/kaggle/working/hi")
sys.path.append("/kaggle/working/Trainer")
sys.path.append("/kaggle/working/TTS/TTS")

In [31]:
!sudo ln -sf /usr/local/cuda/lib64/libcusparse.so /usr/local/cuda/lib64/libcusparse.so.12
!sudo ln -sf /usr/local/cuda/lib64/libnvJitLink.so /usr/local/cuda/lib64/libnvJitLink.so.12

In [32]:
!mkdir -p /kaggle/working/TTS/models/v1/hi/fastpitch

In [33]:
!ln -s /kaggle/working/hi/fastpitch/speakers.pth /kaggle/working/TTS/models/v1/hi/fastpitch/speakers.pth

In [37]:
!python3 -m TTS.bin.synthesize --text translated_text \
    --model_path /kaggle/working/hi/fastpitch/best_model.pth \
    --config_path /kaggle/working/hi/fastpitch/config.json \
    --speakers_file_path /kaggle/working/hi/fastpitch/speakers.pth\
    --vocoder_path /kaggle/working/hi/hifigan/best_model.pth \
    --vocoder_config_path /kaggle/working/hi/hifigan/config.json \
    --speaker_idx "male" \
    --out_path "/kaggle/working/ouput.wav"

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


RuntimeError: module was compiled against NumPy C-API version 0x10 (NumPy 1.23) but the running NumPy has C-API version 0xe. Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem.
 > Using model: fast_pitch
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:0.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > 

In [ ]:
!pip install librosa numpy matplotlib praat-parselmouth

In [ ]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import parselmouth

# Load the audio file and detect the sampling rate
def load_audio(audio_path):
    audio, sr = librosa.load(audio_path, sr=None)  # `sr=None` ensures original sampling rate is used
    return audio, sr

# Plot waveform
def plot_waveform(audio, sr, title="Waveform"):
    plt.figure(figsize=(10, 4))
    librosa.display.waveshow(audio, sr=sr)
    plt.title(title)
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.show()

# Extract F0 (pitch) using Parselmouth (Praat)
def extract_pitch(audio_path):
    snd = parselmouth.Sound(audio_path)
    pitch = snd.to_pitch()
    pitch_values = pitch.selected_array['frequency']
    pitch_values[pitch_values == 0] = np.nan  # Ignore unvoiced parts
    return pitch_values

# Plot pitch contour
def plot_pitch(audio_path, title="Pitch Contour"):
    pitch_values = extract_pitch(audio_path)
    plt.figure(figsize=(10, 4))
    plt.plot(pitch_values, label="F0 (Pitch)")
    plt.title(title)
    plt.xlabel("Frame")
    plt.ylabel("Frequency (Hz)")
    plt.legend()
    plt.show()

# Plot mel spectrogram
def plot_mel_spectrogram(audio, sr, n_mels=80, hop_length=512, title="Mel Spectrogram"):
    mel_spec = librosa.feature.melspectrogram(audio, sr=sr, n_mels=n_mels, hop_length=hop_length)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)

    plt.figure(figsize=(10, 4))
    librosa.display.specshow(mel_spec_db, sr=sr, hop_length=hop_length, x_axis='time', y_axis='mel')
    plt.colorbar(format='%+2.0f dB')
    plt.title(title)
    plt.xlabel("Time (s)")
    plt.ylabel("Mel Frequency")
    plt.show()

In [ ]:
audio_path = "/kaggle/working/ouput.wav"  
audio, sr = load_audio(audio_path)
print(f"Detected Sampling Rate: {sr} Hz")

plot_waveform(audio_ta_female, sr, "Generated Audio Waveform")
plot_pitch(audio_path, "Generated Audio Pitch Contour")
plot_mel_spectrogram(audio_ta_female, sr, title="Generated Audio Mel Spectrogram")

In [ ]:
# Plot mel spectrogram
def plot_mel_spectrogram(audio, sr, n_mels=80, hop_length=512, title="Mel Spectrogram"):
    mel_spec = librosa.feature.melspectrogram(audio, sr=sr, n_mels=n_mels, hop_length=hop_length)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)

    plt.figure(figsize=(10, 4))
    librosa.display.specshow(mel_spec_db, sr=sr, hop_length=hop_length, x_axis='time', y_axis='mel')
    plt.colorbar(format='%+2.0f dB')
    plt.title(title)
    plt.xlabel("Time (s)")
    plt.ylabel("Mel Frequency")
    plt.show()

In [ ]:
plot_mel_spectrogram(audio, sr, title="Generated Audio Mel Spectrogram")